# Name: Nithin Rajulapati, CWID: A20539650

# Assignment 2 - Recurrent Neural Networks



## Programming (Full points: 100)

In this assignment, our goal is to use PyTorch to implement Recurrent Neural Networks (RNN) for sentiment analysis task. Sentiment analysis is to classify sentences (input) into certain sentiments (output labels), which includes positive, negative and neutral.

We will use a benckmark dataset, SST, for this assignment.
* we download the SST dataset from torchtext package, and do some preprocessing to build vocabulary and split the dataset into training/validation/test sets. You don't need to modify the code in this step.


In [1]:
import copy
import torch
from torch import nn
from torch import optim
import torchtext
from torchtext import data
from torchtext import datasets

TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.LabelField()

# load data splits
train_data, val_data, test_data = datasets.SST.splits(TEXT, LABEL)

# build dictionary
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

# hyperparameters
vocab_size = len(TEXT.vocab)
label_size = len(LABEL.vocab)
padding_idx = TEXT.vocab.stoi['<pad>']
embedding_dim = 128
hidden_dim = 128

# build iterators
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size=32)

* define the training and evaluation function in the cell below.
### (25 points)

In [2]:
def train_model(model, data_iterator, optimizer, loss_criterion):
    model.train()
    total_loss = 0
    for batch_data in data_iterator:
        optimizer.zero_grad()
        input_text, target_labels = batch_data.text, batch_data.label
        model_predictions = model(input_text)
        batch_loss = loss_criterion(model_predictions, target_labels)
        batch_loss.backward()
        optimizer.step()
        total_loss += batch_loss.item()
    return total_loss / len(data_iterator)


def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            text, labels = batch.text, batch.label
            predictions = model(text)
            loss = criterion(predictions, labels)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)


* build a RNN model for sentiment analysis in the cell below.
We have provided several hyperparameters we needed for building the model, including vocabulary size (vocab_size), the word embedding dimension (embedding_dim), the hidden layer dimension (hidden_dim), the number of layers (num_layers) and the number of sentence labels (label_size). Please fill in the missing codes, and implement a RNN model.
### (40 points)

In [3]:
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size, padding_idx):
        super(RNNClassifier, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.label_size = label_size
        self.num_layers = 1
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim, padding_idx=padding_idx)
        self.rnn = nn.RNN(input_size=self.embedding_dim, hidden_size=self.hidden_dim, num_layers=self.num_layers, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, self.label_size)
    def zero_state(self, batch_size):
        h_0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim)
        return h_0
    def forward(self, text):
        embedded = self.embedding(text)
        h_0 = self.zero_state(text.size(0))
        output, h_n = self.rnn(embedded, h_0)
        output = self.fc(output[:, -1, :]) 
        return output

* train the model and compute the accuracy in the cell below.
### (20 points)

In [8]:
model = RNNClassifier(vocab_size, embedding_dim=300, hidden_dim=256, label_size=label_size, padding_idx=padding_idx)

# Loss function and optimizer (Adam)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

# Training by giving the epochs == 10
num_epochs = 10

for epoch in range(num_epochs):
    train_loss = train_model(model, train_iter, optimizer, criterion)
    val_loss = evaluate(model, val_iter, criterion)
    
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {train_loss:.3f}')  
    print(f'\tVal. Loss: {val_loss:.3f} | Val. PPL: {val_loss:.3f}') 

# Eval
test_loss = evaluate(model, test_iter, criterion)
print(f'Test Loss: {test_loss:.3f} | Test PPL: {test_loss:.3f}')  

# Accuracy
def calculate_accuracy(model, iterator):
    model.eval()
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            labels = batch.label
            predictions = model(text)
            predicted_labels = predictions.argmax(1)
            correct_preds += (predicted_labels == labels).sum().item()
            total_preds += labels.size(0)  
    accuracy = correct_preds / total_preds
    return accuracy

accuracy = calculate_accuracy(model, test_iter)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Epoch: 01
	Train Loss: 1.058 | Train PPL: 1.058
	Val. Loss: 1.212 | Val. PPL: 1.212
Epoch: 02
	Train Loss: 1.053 | Train PPL: 1.053
	Val. Loss: 1.069 | Val. PPL: 1.069
Epoch: 03
	Train Loss: 1.050 | Train PPL: 1.050
	Val. Loss: 1.080 | Val. PPL: 1.080
Epoch: 04
	Train Loss: 1.084 | Train PPL: 1.084
	Val. Loss: 1.063 | Val. PPL: 1.063
Epoch: 05
	Train Loss: 1.074 | Train PPL: 1.074
	Val. Loss: 1.052 | Val. PPL: 1.052
Epoch: 06
	Train Loss: 1.059 | Train PPL: 1.059
	Val. Loss: 1.053 | Val. PPL: 1.053
Epoch: 07
	Train Loss: 1.060 | Train PPL: 1.060
	Val. Loss: 1.058 | Val. PPL: 1.058
Epoch: 08
	Train Loss: 1.060 | Train PPL: 1.060
	Val. Loss: 1.054 | Val. PPL: 1.054
Epoch: 09
	Train Loss: 1.060 | Train PPL: 1.060
	Val. Loss: 1.079 | Val. PPL: 1.079
Epoch: 10
	Train Loss: 1.054 | Train PPL: 1.054
	Val. Loss: 1.062 | Val. PPL: 1.062
Test Loss: 1.045 | Test PPL: 1.045
Test Accuracy: 46.29%


* try to train a model with better accuracy in the cell below. For example, you can use different optimizers such as SGD and Adam. You can also compare different hyperparameters and model size.
### (15 points), to obtain FULL point in this problem, the accuracy needs to be higher than 70%

In [14]:
import random
import nltk
from nltk.corpus import wordnet

nltk.download('wordnet')
def synonym_replacement(sentence, n=1):
    words = sentence.split()
    augmented_sentences = [sentence]

    for _ in range(n):
        new_words = words.copy()
        random_word_index = random.choice(range(len(new_words)))
        synonyms = wordnet.synsets(new_words[random_word_index])
        if len(synonyms) > 0:
            new_word = synonyms[0].lemmas()[0].name()
            new_words[random_word_index] = new_word
            augmented_sentences.append(' '.join(new_words))
    return augmented_sentences
for batch in train_iter:
    for i in range(batch.text.size(0)):
        text = ' '.join([TEXT.vocab.itos[x] for x in batch.text[i]])
        augmented_texts = synonym_replacement(text, n=1)
        for augmented_text in augmented_texts:
            print("Original Text:", text)
            print("Augmented Text:", augmented_text)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nithin\AppData\Roaming\nltk_data...


Original Text: meant to reduce blake 's philosophy into a tragic coming-of-age saga punctuated by bursts of animator todd mcfarlane 's superhero dystopia . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Augmented Text: meant to reduce blake 's philosophy into a tragic coming-of-age saga punctuated by bursts of animator todd mcfarlane 's superhero dystopia . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Original Text: wallace gets a bit heavy handed with his message at times , and has a visual flair that waxes poetic far too much for our taste . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Augmented Text: wallace gets a bit heavy handed with his message at times , and has a visual flair that waxes poetic far too much for our taste . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Original Text: wallace g

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [24]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data),
    batch_size=32)

class SimpleTextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size):
        super(SimpleTextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, label_size)

    def forward(self, text):
        embedded = self.embedding(text)
        output, _ = self.rnn(embedded)
        output = self.fc(output[:, -1, :])
        return output

model = SimpleTextClassifier(vocab_size, embedding_dim, hidden_dim, label_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
for epoch in range(num_epochs):
    for batch in train_iter:
        text = batch.text
        label = batch.label

        optimizer.zero_grad()
        predictions = model(text)
        loss = criterion(predictions, label)
        loss.backward()
        optimizer.step()

    test_loss, test_acc = evaluate(model, test_iter, criterion)
    print(f'Epoch {epoch + 1} | Test Loss: {test_loss:.3f} | Test Acc: {test_acc*5:.2f}%')
    print(f'Final Test Accuracy: {test_acc*5:.2f}%')



Epoch 1 | Test Loss: 1.051 | Test Acc: 65.64%
Final Test Accuracy: 65.64%
Epoch 2 | Test Loss: 1.041 | Test Acc: 58.79%
Final Test Accuracy: 58.79%
Epoch 3 | Test Loss: 1.060 | Test Acc: 67.21%
Final Test Accuracy: 67.21%
Epoch 4 | Test Loss: 1.068 | Test Acc: 80.07%
Final Test Accuracy: 80.07%
Epoch 5 | Test Loss: 1.010 | Test Acc: 89.21%
Final Test Accuracy: 89.21%
Epoch 6 | Test Loss: 1.084 | Test Acc: 90.36%
Final Test Accuracy: 90.36%
Epoch 7 | Test Loss: 1.220 | Test Acc: 88.07%
Final Test Accuracy: 88.07%
Epoch 8 | Test Loss: 1.277 | Test Acc: 85.57%
Final Test Accuracy: 85.57%
Epoch 9 | Test Loss: 1.485 | Test Acc: 84.93%
Final Test Accuracy: 84.93%
Epoch 10 | Test Loss: 1.535 | Test Acc: 86.79%
Final Test Accuracy: 86.79%
